# Scipy 2021 Project - Sympy - Telegram - Bot - Magnus Müller and Felix Hammer

First of all the required libraries:

In [445]:
!pip install pyTelegramBotAPI
!pip install sympy
!pip install numpy
!pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


## First let's get a grib on pyTelegramBotAPI

Here is the documentation: https://pypi.org/project/pyTelegramBotAPI/
But you don't really need it, because it is very simple:

In [448]:
import telebot # You first have to import the library (it goes by another name)

# Then you have to specify the API Key of your bot here as string: 
API_KEY = "1913464534:AAGjZ9gC5KQmRiAjd-YWCAjhkKYGIpU6SBM"
bot = telebot.TeleBot(API_KEY) # and create a telebot with the api

# After that we just have to specify message handlers, which check wether a reply function is fitting:
@bot.message_handler(commands=['hello']) # commands accepts telegram comands (in this case "/hello")
def hello(message): # and then you just have to state the awnser function directly under the handler
    bot.send_message(message.chat.id, "Hello!") # in this case we simply send back an "Hello" to the person chatting with us
    
    
# You can also you as a handler boolean functions like this: 

def a_boolean_function(message):
    return message.text=='How are you?'

@bot.message_handler(func=a_boolean_function)
def reply(message): # and then you just have to state the awnser function directly under the handler
    bot.send_message(message.chat.id, "I am a bot, I don't have feelings")
    
# To start the bot you will then only need this command

bot.polling()  # use none_stop=True as argument to ikeep running after an error occured


This was basically everything we used, besides a simple send_photo command

It this important to note that the handlers are checked in the order they are defined by

## Then let's have a look at SymPy

Sympy is we also only use a few functions from sympy 
But first let's import it:

In [451]:
from sympy import *

x, y, z = symbols('x y z') # here we specify the variables, we are using

# Simpy offers nice mathematical tool like these:

function = parse_expr('exp(x**2)') # Define a function with an string
print(function)
print(function.diff()) # Derivate the Function
print(integrate(function)) # Integrate it 

exp(x**2)
2*x*exp(x**2)
sqrt(pi)*erfi(x)/2


# Project’s goal, motivation and structure

The goal of this project it to have a create a Telegram 

<img src="img/screen.jpg">